In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

# Ch.09 데이터 수집과 그룹 연산
- key를 이용하여 객체를 여러조각으로 나누기
- 요약통계
- column에 함수 적용
- 피벗 테이블, 교차 일람표
- 변위치 분석

# 9.1 Groupby
- 분리 -> 적용 -> 결합
- group의 기준: 주로 범주형 데이터(categorical data)

In [2]:
df = pd.DataFrame({'key1': list('aabba'),
                  'key2' : list('one,two,one,two,one'.split(',')),
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.052566,0.555699
1,a,two,-0.125928,-1.262205
2,b,one,0.009081,-0.306677
3,b,two,1.049522,0.314296
4,a,one,0.600422,0.566896


groupby 객체 생성 : 연산을 수행하기 전 준비상태(setting)

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.509020
b    0.529302
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.826494
      two    -0.125928
b     one     0.009081
      two     1.049522
Name: data1, dtype: float64

In [6]:
means.unstack()
#dataframe 형태로 변환

key2,one,two
key1,,
a,0.826494,-0.125928
b,0.009081,1.049522


In [7]:
states = np.array(['ohio','california','california','ohio','ohio'])
years = np.array([2005,2005,2006,2005,2006])

DataFrame 안에 존재하지 않는 group key로 groupby 연산

In [8]:
df['data1'].groupby([states, years]).mean()
#같은 길이를 갖는 group key라면 DataFrame 안에 존재하지 않는 외부 group key로도 그룹화 가능

california  2005   -0.125928
            2006    0.009081
ohio        2005    1.051044
            2006    0.600422
Name: data1, dtype: float64

연산의 결과값은 nemerical인 열이나 행만 보여준다. 

In [9]:
df

,key1,key2,data1,data2
0,a,one,1.052566,0.555699
1,a,two,-0.125928,-1.262205
2,b,one,0.009081,-0.306677
3,b,two,1.049522,0.314296
4,a,one,0.600422,0.566896


In [10]:
df.groupby('key1').mean()
#key2 는 numeric type이 아니므로 연산에서 제외

,data1,data2
key1,,
a,0.509020,-0.046537
b,0.529302,0.003809


In [11]:
#범주형 데이터의 경우 각 경우를 count 하는 일이 많다.
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

# 9.1.1 그룹 간 순회하기
- iteration 지원 : 튜플로 반환 (그룹이름, 매칭되는 데이터)
- groupby 객체 : [그룹 key에 해당하는 이름],[그에 매칭되는 데이터] 로 나눌 수 있다.

In [12]:
df

,key1,key2,data1,data2
0,a,one,1.052566,0.555699
1,a,two,-0.125928,-1.262205
2,b,one,0.009081,-0.306677
3,b,two,1.049522,0.314296
4,a,one,0.600422,0.566896


In [13]:
for name, group in df.groupby('key1'):
    print(name)
    print('-'*35)
    print(group)
    print('='*35)

a
-----------------------------------
  key1 key2     data1     data2
0    a  one  1.052566  0.555699
1    a  two -0.125928 -1.262205
4    a  one  0.600422  0.566896
b
-----------------------------------
  key1 key2     data1     data2
2    b  one  0.009081 -0.306677
3    b  two  1.049522  0.314296


In [14]:
for (k1, k2), group_data in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group_data)
    
#return tuple

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.052566  0.555699
4    a  one  0.600422  0.566896
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.125928 -1.262205
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.009081 -0.306677
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.049522  0.314296


data type별로 그룹화하기
- dataframe객체.groupby(df.dtype) : group key를 data type으로 준다

In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [16]:
grouped2 = df.groupby(df.dtypes, axis=1)
grouped2

In [17]:
for (groupname, data) in grouped2:
    print(groupname)
    print('-'*30)
    print(data)
    print('='*30)

float64
------------------------------
      data1     data2
0  1.052566  0.555699
1 -0.125928 -1.262205
2  0.009081 -0.306677
3  1.049522  0.314296
4  0.600422  0.566896
object
------------------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [18]:
dict(list(grouped2))

{dtype('float64'):       data1     data2
 0  1.052566  0.555699
 1 -0.125928 -1.262205
 2  0.009081 -0.306677
 3  1.049522  0.314296
 4  0.600422  0.566896, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

# 9.1.2 칼럼 또는 칼럼의 일부만 선택하기
groupby 객체를 column 이름이 담긴 배열로 indexing
- dataframe객체.groupby(key값들)[인자]
    + 인자가 리스트나 배열: DataFrameGroupby 객체 생성
    + 인자가 단일 값 : SeriesGroupby 객체 생성

In [19]:
df

,key1,key2,data1,data2
0,a,one,1.052566,0.555699
1,a,two,-0.125928,-1.262205
2,b,one,0.009081,-0.306677
3,b,two,1.049522,0.314296
4,a,one,0.600422,0.566896


In [20]:
df['data1'].groupby(df['key1'], axis=0).mean()

key1
a    0.509020
b    0.529302
Name: data1, dtype: float64

# 9.1.3 Dictionary와 Series에서 묶기

In [21]:
#그룹정보가 배열 형태가 아닌 경우
people = pd.DataFrame(np.random.randint(0, 20, (5,5)),
                     columns = list('abcde'),
                     index = 'joe,steve,wes,jim,travis'.split(','))
people

,a,b,c,d,e
joe,17,15,8,8,8
steve,2,19,16,9,2
wes,17,1,5,2,13
jim,4,6,17,14,17
travis,3,2,10,0,1


In [22]:
people.loc[2:3, ['b','c']] = np.nan
people

,a,b,c,d,e
joe,17,15.0,8.0,8,8
steve,2,19.0,16.0,9,2
wes,17,NaN,NaN,2,13
jim,4,6.0,17.0,14,17
travis,3,2.0,10.0,0,1


In [23]:
#groupby (dict형)
mapping = dict(zip(list('abcdef'), ['red','red','blue','blue','red','orange']))
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [24]:
by_col = people.groupby(mapping, axis=1)
by_col.sum()
#joe의 red는 열[a, b, e]의 값. 즉, 첫 번째, 두 번째, 다섯 번째 열의 값
#axis = 1로 했으므로 열에 대해 묶는다. 따라서 범주끼리 연산

,blue,red
joe,16.0,40.0
steve,25.0,23.0
wes,2.0,30.0
jim,31.0,27.0
travis,10.0,6.0


In [25]:
#groupby (series형)
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [26]:
people.groupby(map_series, axis=1).sum()

,blue,red
joe,16.0,40.0
steve,25.0,23.0
wes,2.0,30.0
jim,31.0,27.0
travis,10.0,6.0


# 9.1.4 함수로 묶기
- groupby(함수) : 인자 부분에 함수 반환값을 넣어 그룹화

In [27]:
people

,a,b,c,d,e
joe,17,15.0,8.0,8,8
steve,2,19.0,16.0,9,2
wes,17,NaN,NaN,2,13
jim,4,6.0,17.0,14,17
travis,3,2.0,10.0,0,1


In [28]:
people.groupby(len).sum()
#index( joe, steve, wes...) 들의 문자열 길이를 기준으로 그룹핑

,a,b,c,d,e
3,38,21.0,25.0,24,38
5,2,19.0,16.0,9,2
6,3,2.0,10.0,0,1


In [29]:
people.groupby(len, axis=1).sum()

,1
joe,56.0
steve,48.0
wes,32.0
jim,58.0
travis,16.0


In [30]:
people

,a,b,c,d,e
joe,17,15.0,8.0,8,8
steve,2,19.0,16.0,9,2
wes,17,NaN,NaN,2,13
jim,4,6.0,17.0,14,17
travis,3,2.0,10.0,0,1


In [31]:
np.sign(people.isnull().sum())

a    0
b    1
c    1
d    0
e    0
dtype: int64

In [32]:
people.groupby(np.sign(people.isnull().sum()), axis=1).sum()
# 0은 isnull.sum()이 0이었던 열[a,d,e]의 합

,0,1
joe,33.0,23.0
steve,13.0,35.0
wes,32.0,0.0
jim,35.0,23.0
travis,4.0,12.0


# 9.1.5 색인 단계로 묶기
multiIndex인 index나 column에서 groupby 기준을 찾아서 연산
- dataframe객체.groupby(계층적 색인): level 인자를 통해 처리

In [33]:
col = pd.MultiIndex.from_arrays([['us','us','us','kr','kr'],[1,3,5,1,3]],
                              names = ['city','tenor'])
col

MultiIndex([('us', 1),
            ('us', 3),
            ('us', 5),
            ('kr', 1),
            ('kr', 3)],
           names=['city', 'tenor'])

In [34]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns=col)
hier_df

city         us                            kr          
tenor         1         3         5         1         3
0      1.945592 -0.267427  1.273781  0.686651  1.010485
1     -0.236552  1.004499 -1.541586  0.972686 -1.645311
2      1.286234  0.124413  0.362028 -0.667627  0.486974
3     -0.507151  0.266641 -1.194185 -0.272920  0.851977

In [35]:
hier_df.groupby(level='city', axis=1).mean()

city,kr,us
0,0.848568,0.983982
1,-0.336312,-0.257880
2,-0.090327,0.590891
3,0.289528,-0.478231


# 9.2 데이터 수집
1. 배열 -> 스칼라(mean,sum,count,min,max)
    - dataframe객체.groupby().사용하고 싶은 함수
2. 사용자가 정의한 함수 사용
    - groupby().agg(정의한 함수) : agg 메서드는 df에 함수를 적용하게 해준다.
3. 최적화된 groupby 메서드
    - count : na 값이 아닌 수
    - sum
    - mean
    - median
    - std, var
    - min, max
    - prod : 값의 곱
    - first, last : 첫 번째 값과 마지막 값

In [36]:
df

,key1,key2,data1,data2
0,a,one,1.052566,0.555699
1,a,two,-0.125928,-1.262205
2,b,one,0.009081,-0.306677
3,b,two,1.049522,0.314296
4,a,one,0.600422,0.566896


In [37]:
df[['data1','data2']].groupby(df['key1']).quantile(0.9)
# a를 가진 데이터를 한 그룹으로 묶었을 때 그 데이터들 중 quantile 0.9 인 값
# b를 가진 데이터를 한 그룹으로 묶었을 때 그 데이터들 중 quantile 0.9 인 값

,data1,data2
key1,,
a,0.962137,0.564657
b,0.945478,0.252199


In [38]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [39]:
#grouped = df['data1'].groupby(df['key1'])
grouped.agg(peak_to_peak)

key1
a    1.178494
b    1.040441
Name: data1, dtype: float64

# 9.2.1 칼럼에 여러가지 함수 적용하기
1. 모든 칼럼에 여러 함수 적용
    - dataframe객체.agg([func1, func2,...])
    - dataframe객체.agg([(결과 반환 열 이름, func1), ( , ),,,])
2. 칼럼마다 다른 함수 적용 : 딕셔너리로 {'열': 적용함수}
    - dataframe객체.agg({'열이름' : [func1], ...})
    - dataframe객체.agg({'열이름' : [(결과 반환 열 이름, func1), ( , ) ,,,], '열이름2' : [ ...] , ... )

In [40]:
import seaborn as sns

In [41]:
train = sns.load_dataset('titanic')
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 63.0+ KB


In [43]:
g_train = train[['fare', 'age']].groupby(train['embarked'], axis=0)
g_train

In [44]:
g_train.agg(['mean', 'std', peak_to_peak]).head(6)

fare                                age                        
               mean        std peak_to_peak       mean        std peak_to_peak
embarked                                                                      
C         59.954144  83.912994     508.3167  30.814769  15.434860        70.58
Q         13.276030  14.188047      83.2500  28.089286  16.915396        68.50
S         27.079812  35.887993     263.0000  29.445397  14.143192        79.33

In [45]:
#(원하는 칼럼 이름, 적용하는 함수) 튜플을 인자로 넣어줌
g_train.agg([('평균', 'mean'), ('최댓값', 'max'), ('최솟값','min')])

fare                          age            
                 평균       최댓값     최솟값         평균   최댓값   최솟값
embarked                                                    
C         59.954144  512.3292  4.0125  30.814769  71.0  0.42
Q         13.276030   90.0000  6.7500  28.089286  70.5  2.00
S         27.079812  263.0000  0.0000  29.445397  80.0  0.67

In [46]:
func = [('평균', 'mean'), ('최댓값', 'max'), ('중앙값','median')]

g_train_res = g_train.agg(func)
g_train_res

fare                         age            
                 평균       최댓값    중앙값         평균   최댓값   중앙값
embarked                                                   
C         59.954144  512.3292  29.70  30.814769  71.0  29.0
Q         13.276030   90.0000   7.75  28.089286  70.5  27.0
S         27.079812  263.0000  13.00  29.445397  80.0  28.0

In [47]:
g_train_res['age']

,평균,최댓값,중앙값
embarked,,,
C,30.814769,71.0,29.0
Q,28.089286,70.5,27.0
S,29.445397,80.0,28.0


In [48]:
# 정해진 열에 적용할 합수. dict 형태로 전달
g_train.agg({'fare': ['mean', 'std'], 'age':['max','min']})

fare              age      
               mean        std   max   min
embarked                                  
C         59.954144  83.912994  71.0  0.42
Q         13.276030  14.188047  70.5  2.00
S         27.079812  35.887993  80.0  0.67

In [49]:
g_train.agg({'fare':[('평균값','mean'),('분산','std')], 
             'age':[('최댓값','max'),('최솟값', 'min')]})

fare              age      
                평균값         분산   최댓값   최솟값
embarked                                  
C         59.954144  83.912994  71.0  0.42
Q         13.276030  14.188047  70.5  2.00
S         27.079812  35.887993  80.0  0.67

# 9.2.2 색인되지 않은 집계된 데이터 반환하기
default : 그룹화 후 결과에서 그룹 기준 key는 인덱스로 간다.

groupby 매서드로 묶은 후 묶어진 인덱스를 사용하지 않고 새로운 인덱스를 사용하고 싶을 때
    - dataframe객체.groupby( , as_index =False) 사용

In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 63.0+ KB


In [51]:
train.groupby(['pclass','deck']).mean().head()
#default : groupby의 key가 index로 간다

survived        age     sibsp     parch        fare  adult_male  \
pclass deck                                                                    
1      A     0.466667  44.833333  0.133333  0.133333   39.623887    0.866667   
       B     0.744681  34.955556  0.361702  0.574468  113.505764    0.404255   
       C     0.593220  36.086667  0.644068  0.474576  100.151341    0.525424   
       D     0.758621  40.037037  0.482759  0.344828   63.324286    0.448276   
       E     0.720000  40.083333  0.400000  0.320000   55.740168    0.600000   

                alone  
pclass deck            
1      A     0.800000  
       B     0.468085  
       C     0.372881  
       D     0.310345  
       E     0.480000

In [52]:
train.groupby(['pclass','deck'], as_index =False).mean().head()
#인덱스로 설정되지 않고 column으로 정렬되었다.

,pclass,deck,survived,age,sibsp,parch,fare,adult_male,alone
0,1,A,0.466667,44.833333,0.133333,0.133333,39.623887,0.866667,0.800000
1,1,B,0.744681,34.955556,0.361702,0.574468,113.505764,0.404255,0.468085
2,1,C,0.593220,36.086667,0.644068,0.474576,100.151341,0.525424,0.372881
3,1,D,0.758621,40.037037,0.482759,0.344828,63.324286,0.448276,0.310345
4,1,E,0.720000,40.083333,0.400000,0.320000,55.740168,0.600000,0.480000


# 9.3 그룹별 연산과 변형

In [53]:
df

,key1,key2,data1,data2
0,a,one,1.052566,0.555699
1,a,two,-0.125928,-1.262205
2,b,one,0.009081,-0.306677
3,b,two,1.049522,0.314296
4,a,one,0.600422,0.566896


In [54]:
k1_mean= df.groupby('key1').mean().add_prefix('평균_')
k1_mean

,평균_data1,평균_data2
key1,,
a,0.509020,-0.046537
b,0.529302,0.003809


In [55]:
pd.merge(df, k1_mean, left_on = 'key1', right_index = True)

,key1,key2,data1,data2,평균_data1,평균_data2
0,a,one,1.052566,0.555699,0.509020,-0.046537
1,a,two,-0.125928,-1.262205,0.509020,-0.046537
4,a,one,0.600422,0.566896,0.509020,-0.046537
2,b,one,0.009081,-0.306677,0.529302,0.003809
3,b,two,1.049522,0.314296,0.529302,0.003809


In [56]:
people

,a,b,c,d,e
joe,17,15.0,8.0,8,8
steve,2,19.0,16.0,9,2
wes,17,NaN,NaN,2,13
jim,4,6.0,17.0,14,17
travis,3,2.0,10.0,0,1


In [57]:
key = ['one', 'two', 'one', 'two', 'one']

In [58]:
people.groupby(key).mean()

,a,b,c,d,e
one,12.333333,8.5,9.0,3.333333,7.333333
two,3.000000,12.5,16.5,11.500000,9.500000


In [59]:
people.groupby(key, axis=1).mean()

,one,two
joe,11.000000,11.5
steve,6.666667,14.0
wes,15.000000,2.0
jim,12.666667,10.0
travis,4.666667,1.0


In [60]:
people.transform([np.exp, np.sqrt])

a                       b                       c  \
                 exp      sqrt           exp      sqrt           exp   
joe     2.415495e+07  4.123106  3.269017e+06  3.872983  2.980958e+03   
steve   7.389056e+00  1.414214  1.784823e+08  4.358899  8.886111e+06   
wes     2.415495e+07  4.123106           NaN       NaN           NaN   
jim     5.459815e+01  2.000000  4.034288e+02  2.449490  2.415495e+07   
travis  2.008554e+01  1.732051  7.389056e+00  1.414214  2.202647e+04   

                             d                       e            
            sqrt           exp      sqrt           exp      sqrt  
joe     2.828427  2.980958e+03  2.828427  2.980958e+03  2.828427  
steve   4.000000  8.103084e+03  3.000000  7.389056e+00  1.414214  
wes          NaN  7.389056e+00  1.414214  4.424134e+05  3.605551  
jim     4.123106  1.202604e+06  3.741657  2.415495e+07  4.123106  
travis  3.162278  1.000000e+00  0.000000  2.718282e+00  1.000000

# 9.3.1 apply: 분리 ->적용->병합
- aggregate, transform 함수 : 엄격한 요구사항을 갖는 '특수한 목적의 함수'
- apply 함수 : 일반적인 목적을 갖는 함수
    + apply(함수, 함수의 인자들 설정)
        + 예를 들면 func(a,b,c) 라는 함수가 있다면 apply (fun, a= ,b=, c=)로 함수의 옵션을 넘겨주어 apply 메서드 사용 가능

In [61]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [62]:
train.groupby(['embarked','deck']).mean()

survived    pclass        age     sibsp     parch        fare  \
embarked deck                                                                  
C        A     0.571429  1.000000  50.857143  0.285714  0.000000   38.357743   
         B     0.772727  1.000000  35.476190  0.363636  0.590909  145.964018   
         C     0.666667  1.000000  33.842105  0.428571  0.380952   98.582533   
         D     0.846154  1.153846  39.692308  0.538462  0.230769   74.470208   
         E     1.000000  1.000000  35.000000  0.600000  0.400000   92.905840   
         F     1.000000  3.000000        NaN  1.000000  1.000000   22.358300   
         G          NaN       NaN        NaN       NaN       NaN         NaN   
Q        A          NaN       NaN        NaN       NaN       NaN         NaN   
         B          NaN       NaN        NaN       NaN       NaN         NaN   
         C     0.500000  1.000000  38.500000  1.500000  0.000000   90.000000   
         D          NaN       NaN        NaN       NaN       NaN         NaN   
         E     1.000000  2.000000        NaN  0.000000  0.000000   12.350000   
         F     0.000000  3.000000        NaN  0.000000  0.000000    7.750000   
         G          NaN       NaN        NaN       NaN       NaN         NaN   
S        A     0.375000  1.000000  36.400000  0.000000  0.250000   40.731763   
         B     0.695652  1.000000  33.090909  0.391304  0.608696   85.372283   
         C     0.555556  1.000000  37.347333  0.722222  0.555556  101.630442   
         D     0.700000  1.100000  38.555556  0.350000  0.350000   46.047915   
         E     0.692308  1.346154  38.740000  0.269231  0.307692   38.306731   
         F     0.636364  2.272727  19.954545  0.545455  0.545455   19.359091   
         G     0.500000  3.000000  14.750000  0.500000  1.250000   13.581250   

               adult_male     alone  
embarked deck                        
C        A       0.857143  0.714286  
         B       0.454545  0.409091  
         C       0.476190  0.380952  
         D       0.384615  0.307692  
         E       0.200000  0.400000  
         F       0.000000  0.000000  
         G            NaN       NaN  
Q        A            NaN       NaN  
         B            NaN       NaN  
         C       0.500000  0.000000  
         D            NaN       NaN  
         E       0.000000  1.000000  
         F       1.000000  1.000000  
         G            NaN       NaN  
S        A       0.875000  0.875000  
         B       0.391304  0.478261  
         C       0.555556  0.388889  
         D       0.500000  0.450000  
         E       0.576923  0.538462  
         F       0.363636  0.545455  
         G       0.000000  0.000000

# 9.3.2 변위치 분석과 버킷 분석
groupby와 cut의 조합

cut으로 나눈 구간을 groupby의 인자로 넣어준다.

In [63]:
frame = pd.DataFrame({'data1' : np.random.randn(1000),
                     'data2' : np.random.randn(1000)})
frame.head()

,data1,data2
0,0.173309,0.659048
1,-0.911357,-1.522777
2,-0.498594,-0.246845
3,-0.697718,2.953412
4,-0.078690,-0.744884


In [64]:
factor = pd.cut(frame['data1'], 4)
#split into 4 buckets
#네 개의 카테고리로 나눴다.
factor

0        (0.0731, 1.67]
1      (-1.524, 0.0731]
2      (-1.524, 0.0731]
3      (-1.524, 0.0731]
4      (-1.524, 0.0731]
             ...       
995    (-1.524, 0.0731]
996      (0.0731, 1.67]
997      (0.0731, 1.67]
998    (-1.524, 0.0731]
999    (-1.524, 0.0731]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-3.128, -1.524] < (-1.524, 0.0731] < (0.0731, 1.67] < (1.67, 3.268]]

In [65]:
def get_status(group):
    return{'최소': group.min(), '최대': group.max(), 
           '개수': group.count(), '평균': group.mean()}
grouped = frame['data2'].groupby(factor)
grouped.apply(get_status)

data1               
(-3.128, -1.524]  최소     -1.708363
                  최대      2.335645
                  개수     78.000000
                  평균      0.056151
(-1.524, 0.0731]  최소     -3.485093
                  최대      2.953412
                  개수    475.000000
                  평균      0.014284
(0.0731, 1.67]    최소     -3.041979
                  최대      2.682496
                  개수    406.000000
                  평균     -0.020549
(1.67, 3.268]     최소     -2.465378
                  최대      1.276109
                  개수     41.000000
                  평균     -0.026004
Name: data2, dtype: float64

In [66]:
grouped.apply(get_status).unstack()


,최소,최대,개수,평균
data1,,,,
"(-3.128, -1.524]",-1.708363,2.335645,78.0,0.056151
"(-1.524, 0.0731]",-3.485093,2.953412,475.0,0.014284
"(0.0731, 1.67]",-3.041979,2.682496,406.0,-0.020549
"(1.67, 3.268]",-2.465378,1.276109,41.0,-0.026004


In [67]:
grouping = pd.qcut(frame['data1'], 10)
grouping

0       (-0.0447, 0.179]
1       (-1.404, -0.879]
2       (-0.574, -0.288]
3       (-0.879, -0.574]
4      (-0.288, -0.0447]
             ...        
995     (-0.879, -0.574]
996       (0.443, 0.733]
997       (0.733, 1.124]
998    (-0.288, -0.0447]
999     (-0.574, -0.288]
Name: data1, Length: 1000, dtype: category
Categories (10, interval[float64]): [(-3.1229999999999998, -1.404] < (-1.404, -0.879] < (-0.879, -0.574] < (-0.574, -0.288] ... (0.179, 0.443] < (0.443, 0.733] < (0.733, 1.124] < (1.124, 3.268]]

In [68]:
grouping = pd.qcut(frame['data1'],10, labels = False)
grouping

0      5
1      1
2      3
3      2
4      4
      ..
995    2
996    7
997    8
998    4
999    3
Name: data1, Length: 1000, dtype: int64

In [69]:
grouped2 = frame['data2'].groupby(grouping)
grouped2.apply(get_status).unstack()

,최소,최대,개수,평균
data1,,,,
0,-1.708363,2.814862,100.0,0.038936
1,-3.049789,2.593166,100.0,-0.023277
2,-2.620028,2.953412,100.0,-0.022887
3,-1.838460,2.428416,100.0,-0.005287
4,-3.485093,2.391774,100.0,0.104001
5,-2.625889,2.437613,100.0,0.018618
6,-3.041979,2.682496,100.0,0.000955
7,-1.968821,2.179051,100.0,-0.076927
8,-2.603366,2.565631,100.0,-0.100166


# 9.3.3 예제: 그룹에 국한된 값으로 누락된 값 채우기

1. 기존: dropna(how='all'/'any'), fillna
2. 그룹별로 채우고 싶은 값이 다른 경우
    - 데이터를 그룹으로 나눈다 -> apply를 이용하여 각 그룹에 fillna 적용

In [73]:
train.loc[3:100:3, ['pclass']] = np.nan

In [74]:
train['pclass'].isnull().sum()

33

In [80]:
train['pclass'].fillna(method = 'ffill', inplace=True)

In [81]:
train['pclass'].isnull().sum()

0

In [82]:
Pcls_train = train.groupby(train['pclass'])

In [83]:
group_fillmean = lambda g : g.fillna(g.mean())

In [84]:
no_nan_train = Pcls_train.apply(group_fillmean)

In [86]:
istherenull = []

for i in no_nan_train.columns:
    a = (True in no_nan_train[i].isnull().values)
    istherenull.append(a)

In [88]:
no_nan_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null float64
sex            891 non-null object
age            891 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(3), int64(3), object(5)
memory usage: 63.0+ KB


In [87]:
istherenull
#embarked, deck, embark_town 을 제외한 모든 열이 Pclass를 기준으로
##한 그룹의 평균값으로 nan 값이 채워졌다.

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False]

# 9.3.4 예제 : 랜덤 표본과 순열
효과적으로 표본 추출하기 : np.random.permutation(N) : 처음 K원소 선택

N : 전체 데이터셋 크기

K : 원하는 표본 크기

In [90]:
arr = [i for i in range(100)]
arr

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [92]:
np.random.permutation(arr)[:int(len(arr)*0.6)]

array([13, 59, 55, 37, 77,  9, 63, 64, 18, 78, 41, 97, 62, 27, 96, 83, 60,
       82, 10, 29, 25, 67, 85, 61, 71,  7, 56, 40, 86, 33, 45, 12, 91,  3,
       50, 73, 24, 89, 94, 17, 28, 95, 93, 20,  1, 34, 43, 75,  8, 69,  4,
       38, 39, 98, 32, 57, 58, 49, 80, 22])